In [ ]:
import pandas as pd
from tqdm import tqdm
import requests
API_KEY = ""

In [27]:
corp_list_df = pd.read_csv("./data/EdinetcodeDlInfo_utf8.csv", encoding="utf-8", skiprows=1)
corp_list_df.dropna(subset=["提出者法人番号"], inplace=True)
corp_list_df["提出者法人番号"] = corp_list_df["提出者法人番号"].astype(int).astype(str)

In [ ]:
corp_list_df.columns
# Index(['ＥＤＩＮＥＴコード', '提出者種別', '上場区分', '連結の有無', '資本金', '決算日', '提出者名', '提出者名（英字）',
#        '提出者名（ヨミ）', '所在地', '提出者業種', '証券コード', '提出者法人番号'],
#       dtype='object')

Index(['ＥＤＩＮＥＴコード', '提出者種別', '上場区分', '連結の有無', '資本金', '決算日', '提出者名', '提出者名（英字）',
       '提出者名（ヨミ）', '所在地', '提出者業種', '証券コード', '提出者法人番号'],
      dtype='object')

In [32]:
corp_num_goldman_sachs = "9010401059056" # ゴールドマン・サックス
corp_num_sumitomo = "1010001008692" # 住友商事

In [30]:
def get_corp_docs_list(corp_num: str):
    if len(corp_num) != 13:
        raise ValueError("Corporation number must be 13 digits long.")
    corp_record = corp_list_df[corp_list_df["提出者法人番号"] == corp_num]
    if corp_record.empty:
        raise ValueError("Corporation number not found in the list.")
    return corp_record

In [33]:
get_corp_docs_list(corp_num_goldman_sachs)

,ＥＤＩＮＥＴコード,提出者種別,上場区分,連結の有無,資本金,決算日,提出者名,提出者名（英字）,提出者名（ヨミ）,所在地,提出者業種,証券コード,提出者法人番号
4452,E11198,内国法人・組合（有価証券報告書等の提出義務者以外）,NaN,NaN,83616.0,NaN,ゴールドマン・サックス証券株式会社,NaN,ゴールドマンサックスショウケンカブシキガイシャ,港区虎ノ門二丁目６番１号虎ノ門ヒルズステーションタワー,内国法人・組合（有価証券報告書等の提出義務者以外）,NaN,9010401059056


In [34]:
get_corp_docs_list(corp_num_sumitomo)

,ＥＤＩＮＥＴコード,提出者種別,上場区分,連結の有無,資本金,決算日,提出者名,提出者名（英字）,提出者名（ヨミ）,所在地,提出者業種,証券コード,提出者法人番号
1746,E02528,内国法人・組合,上場,有,221023.0,3月31日,住友商事株式会社,SUMITOMO CORPORATION,スミトモショウジカブシキガイシャ,千代田区大手町２丁目３番２号,卸売業,80530,1010001008692


In [69]:
url_template = "https://api.edinet-fsa.go.jp/api/v2/documents.json?date={date}&type=2&Subscription-Key={api_key}"

def get_document_list_by_date(date: str, api_key: str = API_KEY):
    """
    Get a list of documents submitted on a specific date.

    :param date: Date in the format YYYY-MM-DD
    :param api_key: API key for authentication
    :return: JSON response containing the list of documents
    """
    url = url_template.format(date=date, api_key=api_key)
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Error fetching data: {response.status_code} - {response.text}")
    data = response.json()
    if data["metadata"]["status"] != "200":
        print(url)
        print(data)
        raise
    try:
        count = data["metadata"]["resultset"]["count"]
        results = data["results"]
    except KeyError as e:
        print(url)
        print(data)
        raise e
    df = pd.DataFrame(results)
    assert len(df) == count, "Count mismatch between metadata and results"
    return df

In [61]:
documents_list_df = get_document_list_by_date("2025-03-31")
documents_list_df.dropna(subset=["JCN"], inplace=True)
documents_list_df["JCN"] = documents_list_df["JCN"].astype(int).astype(str)
documents_list_df.head()

https://api.edinet-fsa.go.jp/api/v2/documents.json?date=2025-03-31&type=2&Subscription-Key=89d5ae7deb7d466f8cb5a501f6211e71


,seqNumber,docID,edinetCode,secCode,JCN,filerName,fundCode,ordinanceCode,formCode,docTypeCode,...,opeDateTime,withdrawalStatus,docInfoEditStatus,disclosureStatus,xbrlFlag,pdfFlag,attachDocFlag,englishDocFlag,csvFlag,legalStatus
0,1,S100VIG2,E01267,63260,1120001037978,株式会社 クボタ,None,010,110000,080,...,None,0,0,0,1,1,1,0,1,1
1,2,S100VD6D,E32992,39810,3010401109114,株式会社ビーグリー,None,010,053000,180,...,None,0,0,0,1,1,0,0,1,1
2,3,S100VFS0,E09666,None,9010001062807,アムンディ・ジャパン株式会社,G15482,030,04A000,030,...,None,0,0,0,1,1,1,0,1,1
3,4,S100VFKJ,E04730,None,7140001041411,株式会社サイプレスクラブ,None,010,030000,120,...,None,0,0,0,1,1,1,0,1,1
4,5,S100VIX1,E05116,47760,5010001072207,サイボウズ株式会社,None,010,053000,180,...,None,0,0,0,1,1,0,0,1,1


In [72]:
# # 2024/11 のデータを全て取得してキャッシュ
# for day in tqdm(range(1, 31)):
#     date_str = f"2024-11-{day:02d}"
#     try:
#         df = get_document_list_by_date(date_str)
#         df.to_csv(f"cache/documents_list_{date_str}.csv", index=False, encoding="utf-8")
#     except Exception as e:
#         print(f"Failed to fetch data for {date_str}: {e}")

In [71]:
# 保存したデータを縦に繋げて保存
dfs = []
for day in tqdm(range(1, 31)):
    date_str = f"2024-11-{day:02d}"
    # ファイルが存在して中身が空でない場合のみ読み込む
    try:
        with open(f"cache/documents_list_{date_str}.csv", "r", encoding="utf-8") as f:
            if f.read().strip():  # Check if file is not empty
                pass
            else:
                continue  # Skip empty files
    except FileNotFoundError:
        continue  # Skip if file does not exist
    except Exception as e:
        print(f"Error reading file for {date_str}: {e}")
        continue
    df = pd.read_csv(f"cache/documents_list_{date_str}.csv", encoding="utf-8")
    dfs.append(df)

if dfs:
    all_documents_df = pd.concat(dfs, ignore_index=True)
    all_documents_df.to_csv("cache/documents_list_2024-11.csv", index=False, encoding="utf-8")

100%|██████████| 30/30 [00:00<00:00, 468.49it/s]


In [73]:
documents_list_df = pd.read_csv("cache/documents_list_2024-11.csv", encoding="utf-8")
documents_list_df.dropna(subset=["JCN"], inplace=True)
documents_list_df["JCN"] = documents_list_df["JCN"].astype(int).astype(str)
documents_list_df.head()

,seqNumber,docID,edinetCode,secCode,JCN,filerName,fundCode,ordinanceCode,formCode,docTypeCode,...,opeDateTime,withdrawalStatus,docInfoEditStatus,disclosureStatus,xbrlFlag,pdfFlag,attachDocFlag,englishDocFlag,csvFlag,legalStatus
0,1,S100UKYJ,E01428,79390.0,9240001003119,株式会社研創,NaN,10.0,043A00,160.0,...,NaN,0,0,0,1,1,0,0,1,1
1,2,S100ULJ6,E02174,72560.0,5021001006269,河西工業株式会社,NaN,10.0,053001,190.0,...,NaN,0,0,0,1,1,0,0,1,1
2,3,S100ULUA,E06458,NaN,6010001024529,ドイチェ・アセット・マネジメント株式会社,G06904,30.0,995000,180.0,...,NaN,0,0,0,1,1,0,0,1,1
3,4,S100UKYM,E01428,79390.0,9240001003119,株式会社研創,NaN,10.0,042000,135.0,...,NaN,0,0,0,0,1,0,0,0,1
4,5,S100UKE0,E12566,NaN,2010001066780,イーストスプリング・インベストメンツ株式会社,G11694,30.0,995000,180.0,...,NaN,0,0,0,1,1,0,0,1,1


In [ ]:
# 住友商事のドキュメントを抽出
sumitomo_docs_df = documents_list_df[documents_list_df["JCN"] == corp_num_sumitomo]
sumitomo_docs_df

,seqNumber,docID,edinetCode,secCode,JCN,filerName,fundCode,ordinanceCode,formCode,docTypeCode,...,opeDateTime,withdrawalStatus,docInfoEditStatus,disclosureStatus,xbrlFlag,pdfFlag,attachDocFlag,englishDocFlag,csvFlag,legalStatus
706,118,S100UMNK,E02528,80530.0,1010001008692,住友商事株式会社,NaN,10.0,043A00,160.0,...,NaN,0,0,0,1,1,0,0,1,1
708,120,S100UMNL,E02528,80530.0,1010001008692,住友商事株式会社,NaN,10.0,042000,135.0,...,NaN,0,0,0,0,1,0,0,0,1
8712,107,S100UTNI,E02528,80530,1010001008692,住友商事株式会社,NaN,60.0,010002,350.0,...,NaN,0,0,0,1,1,0,0,1,1


In [ ]:
import zipfile
import io

def get_document(docID: str, api_key: str = API_KEY):
    """
    Get the document content by its ID.

    :param docID: Document ID
    :param api_key: API key for authentication
    :return: JSON response containing the document content
    """
    url = f"https://disclosure.edinet-fsa.go.jp/api/v2/documents/{docID}?type=5&Subscription-Key={api_key}"
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Error fetching document: {response.status_code} - {response.text}")

    # zipデータが送られてくるので解凍して中のファイル名を取得する
    zip_bytes = io.BytesIO(response.content)
    df_dict = {}
    with zipfile.ZipFile(zip_bytes) as zf:
        file_names = zf.namelist()
        # csv ファイルなら df_dict に追加
        for name in file_names:
            if name.lower().endswith(".csv"):
                with zf.open(name) as f:
                    raw = f.read()
                    df_dict[name] = pd.read_csv(io.BytesIO(raw), encoding="utf-16", sep="\t")

    return df_dict

In [107]:
def get_documents_by_corp_num(corp_num: str, api_key: str = API_KEY):
    corp_record = documents_list_df[documents_list_df["JCN"] == corp_num]
    if corp_record.empty:
        raise ValueError("Corporation number not found in the documents list.")
    doc_ids = corp_record["docID"].tolist()
    print(doc_ids)
    documents = {}
    for doc_id in tqdm(doc_ids, desc=f"Fetching documents for {corp_num}"):
        try:
            doc_content = get_document(doc_id, api_key)
            documents[doc_id] = doc_content
        except Exception as e:
            print(f"Failed to fetch document {doc_id}: {e}")
    return documents

"3120001136159"
docs = get_documents_by_corp_num("3120001136159")

['S100UVDZ', 'S100UVE1', 'S100UVE4']


Fetching documents for 3120001136159: 100%|██████████| 3/3 [00:00<00:00, 12.99it/s]

Failed to fetch document S100UVE4: File is not a zip file


In [109]:
def find_jpcrp_docs(docs):
    for doc_id, content in docs.items():
        for file_name, df in content.items():
            if "jpcrp" in file_name.lower():
                return df
find_jpcrp_docs(docs)

,要素ID,項目名,コンテキストID,相対年度,連結・個別,期間・時点,ユニットID,単位,値
0,jpdei_cor:NumberOfSubmissionDEI,提出回数、DEI,FilingDateInstant,提出日時点,その他,時点,pure,NaN,1
1,jpcrp-esr_cor:DocumentTitleCoverPage,提出書類、表紙,FilingDateInstant,提出日時点,その他,時点,－,－,臨時報告書
2,jpcrp-esr_cor:PlaceOfFilingCoverPage,提出先、表紙,FilingDateInstant,提出日時点,その他,時点,－,－,近畿財務局長
3,jpcrp-esr_cor:FilingDateCoverPage,提出日、表紙,FilingDateInstant,提出日時点,その他,時点,－,－,2024-11-29
4,jpcrp-esr_cor:CompanyNameCoverPage,会社名、表紙,FilingDateInstant,提出日時点,その他,時点,－,－,サントリーホールディングス株式会社
5,jpcrp-esr_cor:CompanyNameInEnglishCoverPage,英訳名、表紙,FilingDateInstant,提出日時点,その他,時点,－,－,Suntory Holdings Limited
6,jpcrp-esr_cor:TitleAndNameOfRepresentativeCove...,代表者の役職氏名、表紙,FilingDateInstant,提出日時点,その他,時点,－,－,代表取締役社長 新浪 剛史
7,jpcrp-esr_cor:AddressOfRegisteredHeadquarterCo...,本店の所在の場所、表紙,FilingDateInstant,提出日時点,その他,時点,－,－,大阪市北区堂島浜二丁目１番40号
8,jpcrp-esr_cor:TelephoneNumberCoverPage,電話番号、表紙,FilingDateInstant,提出日時点,その他,時点,－,－,０６（６３４６）１６８２
9,jpcrp-esr_cor:NameOfContactPersonCoverPage,事務連絡者氏名、表紙,FilingDateInstant,提出日時点,その他,時点,－,－,常務執行役員 経営企画本部長 宮永 暢
